In [1]:
# import the usual
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import math
import re
import itertools
from scipy.sparse import csr_matrix

%matplotlib inline
pd.set_option('display.max_columns', 500)

In [2]:
import findspark
findspark.init()
#from pyspark.ml.recommendation import ALS
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

In [3]:
sc = SparkContext()

In [4]:
# sp = SparkSession.builder.appName("s").getOrCreate()

In [4]:
data = sc.textFile(r"F:\Data_Repository\lastfm\df_slot1.tsv")

In [5]:
type(data)

pyspark.rdd.RDD

In [6]:
data = data.map(lambda x: x.split('\t'))
# header = data.first()
# print(header)

In [12]:
# data2 = data.map(lambda x : [x[i] for i in [0,1,3]])
data2 = data.map(lambda x : [x[i] for i in [0,1,2]]) #only 3 columns exist

In [13]:
#print ("length of uncleaned data -",data2.count())

In [14]:
data.first()

['user_000001', '15 Step', '2']

In [15]:
data2.first()

['user_000001', '15 Step', '2']

In [16]:
def isNumber(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    return False

In [17]:
data2 = data2.filter(lambda x: isNumber(x[2])) # Remove faulty rows
data2 = data2.map(lambda x: [x[0], x[1], float(x[2])]) #Change plays into float

In [18]:
data2.first()

['user_000001', '15 Step', 2.0]

#### no need to reduce

In [20]:
%%time
#Filter out values with more than 500 plays (for this sake of simplicity)
#print ("length of cleaned data -",data2.count())
#data2 = data2.filter(lambda x : x[2] <= 50)
# print (data2.take(2))
print ("length of slot data -",data2.count())

length of slot data - 247852
Wall time: 1.17 s


In [21]:
%%time
#Convert strings into integers
users = data2.map(lambda x: x[0]).distinct().zipWithIndex()
artists = data2.map(lambda x: x[1]).distinct().zipWithIndex()
# int_user = users.map(lambda u: (u[1], u[0]))
# int_artist = artists.map(lambda i: (i[1], i[0]))
# users.collect()
# artists.collect()

Wall time: 4.76 s


In [22]:
%%time
# Substitutes the ObjectIDs in the ratings RDD with the corresponding int values
data2 = data2.map(lambda r: (r[0], (r[1], r[2]))).join(users).map(lambda r: (r[1][1], r[1][0][0], r[1][0][1]))
data2 = data2.map(lambda r: (r[1], (r[0], r[2]))).join(artists).map(lambda r: (r[1][0][0], r[1][1], r[1][0][1]))

Wall time: 172 ms


In [23]:
%%time
# data2.filter(lambda x: x[0] == 12).collect()
# plays = data2.map(lambda x: x[2]).collect() ##seems like data2 loses data after .collect is called
plays = data2.map(lambda x: x[2])
# data2.collect()

Wall time: 0 ns


In [24]:
%%time
data2.first()

Wall time: 8.55 s


(1, 931, 11.0)

In [25]:
%%time
# Use 'Rating' function to get the values in the right format
data2 = data2.map(lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))
#data2.count()

Wall time: 0 ns


#### Dataframe converted to rating

In [27]:
data2.first()

Rating(user=1, product=931, rating=11.0)

In [28]:
%%time
# Use randomsplit to split the data into train, validation and testing sets

training_RDD, validation_RDD, test_RDD = data2.randomSplit([6, 2, 2], seed = 2)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

Wall time: 0 ns


In [29]:
%%time
# Define computeRMSE

def computeRMSE(model,data):
    
    """ Takes ALS models and testing data as input and returns RMSE value """
    
    data_for_predict = data.map(lambda x: (x[0], x[1]))
    
    predictions = model.predictAll(data_for_predict).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = data.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
    return error

Wall time: 0 ns


In [45]:
%%time
seed = 5
#iterations = 10
iterations = 1
regularization_parameter = 0.1
ranks = [5,10,15]
#ranks = [15] #to reduce loop
errors = [0, 0, 0]
err = 0
tolerance = 0.02
alpha = 0.01

min_error = float('inf')
best_rank = -1
best_iteration = -1

for rank in ranks:
    e = 0
    #for i in range(5):
    for i in range(1):
        # Split the data
        training_RDD, validation_RDD, test_RDD = data2.randomSplit([6, 2, 2], seed = 2)
        validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
        test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))
        
        model = ALS.trainImplicit(training_RDD, rank, seed=seed, iterations=iterations,
                          lambda_=regularization_parameter,alpha=alpha)
    #     predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    #     rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    #     error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
        e += computeRMSE(model,validation_RDD)
    
    error = e/5
    errors[err] = error
    err += 1
    print ('For rank %s the RMSE is %s' % (rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print ('The best model was trained with rank %s' % best_rank)
# For rank 5 the RMSE is 6.67535527624558
# For rank 10 the RMSE is 6.461547082828238
# For rank 15 the RMSE is 6.531429770101694
# The best model was trained with rank 10
# Wall time: 8min 21s

# For rank 5 the RMSE is 1.2219171452924378
# For rank 10 the RMSE is 1.5340979416430192
# For rank 15 the RMSE is 1.2701273441785674
# The best model was trained with rank 5
# Wall time: 1min 33s


For rank 5 the RMSE is 1.2849775735924476
For rank 10 the RMSE is 1.284670193890993
For rank 15 the RMSE is 1.2838877612700377
The best model was trained with rank 15
Wall time: 2min 50s


In [34]:
# %%time
# seed = 5
# iterations = 1
# regularization_parameter = 0.1
# # ranks = [5,10,15]
# ranks = [15] #to reduce loop
# errors = [0, 0, 0]
# err = 0
# tolerance = 0.02
# alpha = 0.01

# min_error = float('inf')
# best_rank = -1
# best_iteration = -1

# for rank in ranks:
#     e = 0
#     for i in range(1):
#         # Split the data
#         training_RDD, validation_RDD, test_RDD = data2.randomSplit([6, 2, 2])
#         validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
#         test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))
        
#         model = ALS.trainImplicit(training_RDD, rank, seed=seed, iterations=iterations,
#                           lambda_=regularization_parameter,alpha=alpha)
#     #     predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
#     #     rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
#     #     error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
#         e += computeRMSE(model,validation_RDD)
    
#     error = e/5
#     errors[err] = error
#     err += 1
#     print ('For rank %s the RMSE is %s' % (rank, error))
#     if error < min_error:
#         min_error = error
#         best_rank = rank

# print ('The best model was trained with rank %s' % best_rank)

# # For rank 15 the RMSE is 1.5317847186327949
# # The best model was trained with rank 15
# # Wall time: 32.5 s

# # For rank 15 the RMSE is 1.2970103120850032
# # The best model was trained with rank 15
# # Wall time: 30.4 s

For rank 15 the RMSE is 1.2970103120850032
The best model was trained with rank 15
Wall time: 30.4 s


In [36]:
%%time
# Final Model

model = ALS.train(data2, best_rank, seed=seed, iterations= iterations,
                      lambda_=regularization_parameter)
predictions = model.predictAll(validation_for_predict_RDD)

Wall time: 8.35 s


In [47]:
## validation rdd
validation_RDD.first()

Rating(user=23, product=931, rating=4.0)

In [48]:
## validation having just user item
validation_for_predict_RDD.first()

(23, 931)

In [37]:
%%time
print (validation_RDD.count())
print (predictions.count())
#computeRMSE(model,data2)

49731
49731
Wall time: 35.9 s


In [44]:
%%time
#computeRMSE(model,data2)

Wall time: 0 ns


In [40]:
%%time
recos = model.recommendProducts(0,20)

Wall time: 202 ms


In [41]:
%%time
scores = []
for i in recos:
     scores.append(float(i[2]))

Wall time: 0 ns


In [42]:
%%time
predictions.filter(lambda x : x[0] == 0).collect()

Wall time: 4.69 s


[Rating(user=0, product=204, rating=0.1302139087676721),
 Rating(user=0, product=288, rating=0.08125482420744135),
 Rating(user=0, product=2508, rating=0.5641149002283767),
 Rating(user=0, product=2856, rating=0.35142433988278265),
 Rating(user=0, product=2820, rating=1.6709564288988954),
 Rating(user=0, product=2808, rating=1.830578520171482),
 Rating(user=0, product=768, rating=1.2665242119013902),
 Rating(user=0, product=2844, rating=0.861640002186876),
 Rating(user=0, product=2472, rating=0.40332568305465166),
 Rating(user=0, product=2652, rating=1.096171466058772),
 Rating(user=0, product=792, rating=-0.6297734465634264),
 Rating(user=0, product=2124, rating=1.1416842288348323),
 Rating(user=0, product=132, rating=0.6341896913551128),
 Rating(user=0, product=240, rating=2.258801205518232),
 Rating(user=0, product=2893, rating=-0.01438448255242264),
 Rating(user=0, product=2089, rating=0.5896484975427381),
 Rating(user=0, product=793, rating=-0.21629310312896077),
 Rating(user=0, p

In [29]:
# Save and load model
model.save(sc, "./model/als_plays_v3_slot1")